In [ ]:
import scipy.constants as constants
from qutip import *
import numpy as np
import pandas as pd
from scipy import interpolate
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import optimize

%matplotlib inline

##### Parameters
$\omega_c$-cavity frequency
<br>
$\omega_f$ - external field frequency
<br>
$\omega_a$ - atom(qubit) frequency 
<br>
$g_0$-coupling coefficent
<br>
$g=\frac{g_0}{\sqrt{1+\frac{\omega_f}{\omega_\Delta}}}$
<br>
$\kappa_c$ - Kerr coefficient (cavity dissipation rate)
<br>
$n_c$ - cavity number operator
<br>
$n_t$ - avg number of thermal bath excitation
<br>
$\gamma_1$-qubit longitudinal damping rate
<br>
$\gamma_2$-qubit transverse damping rate
<br>
$\gamma_c$-cavity mode damping rate
<br>
c_levels-cavity levels
<br>
$\omega_d$-
<br>
$\omega_2$-
<br>
$\omega_1$-
<br>
$\omega_p$-

In [ ]:
params = DefaultParameters()

params.wc = 6.6408              #cavity mode angular freq
params.wa =                     #qubit freq (hbar*wa=energy gap between ground and exited states)
params.w1 =                     #
params.wp = 
params.Dpa=params.wp-params.wa
params.Dpc=params.wp-params.wc

g1 = 0.3355                                          #coupling coefficent
params.g = g0/np.sqrt(1+(params.ff/params.fDelta)**2)

params.p0= -np.tanh((params.hbar*params.wa)/(2*kb*T))

params.kappa = 2*np.pi*6.0e-5

params.n_c = 0.0
params.n_t = 0.0

params.gamma1= 4.07e-5
params.gamma2=
params.gammac=

params.c_levels = 3

#parameters after the tramsformation

params.alpha=np.arctan(-params.Dpa/params.w1)
params.wr=np.sqrt(params.wa**2 + params.Dpa**2)#rabi oscillation
params.gamma1tag=params.gamma2+(params.gamma1-params.gamma2)*(np.sin(params.alpha))**2
params.gamma2tag=0.5*(params.gamma1+params.gamma2)+0.5*(params.gamma1-params.gamma2)*(np.sin(params.alpha))**2
params.g1tag=0.5*np.sin(params.alpha+1)*params.g1
params.p0tag=(np.sin(params.alpha)*params.gamma1(params.p0))/(params.gamma2(1+((params.gamma1-params.gamma2)/params.gamma2)*(np.sin(params.alpha))**2))
#################Global Variables######################
kb=1.38*(10**(-23)) #J/K
T =                 #K

### Hamiltomian 
$$\hbar^{-1}H_0=-\frac{\Delta_{pa}\Sigma_z}{2}+\frac{\omega_1(\Sigma_++\Sigma_-)}{4}-\Delta_{pc}A^{\dagger}A+\frac{g_1(A\Sigma_-+\Sigma_+A^{\dagger})}{2}$$


In [ ]:
def construct_H(params)
    Sz=tensor(qeye(params.c_levels),sigmaz())
    Sm=tensor(qeye(params.c_levels),sigmam())
    a =tensor(destroy(params.c_levels),qeye(2))
    H0=-0.5*(params.wp-params.wa)Sz+0.25*w1*(Sm.dag()+Sm)-(params.wp-params.wc)*a.dag()*a+0.5*params.g*(a*Sm+Sm.dag()*a.dag())
    return H0 

### Bloch Equations 
The bloch equations of motion for the expectation values (derived from Ehrenfest theorem) 

1. $a=<A>$
<br>
2. $p_+=<\Sigma_+>$
<br>
3. $p_z=<\Sigma_z>$
by using 
$$ \frac{d}{dt}<A>=\frac{1}{i\hbar}<[H,A]>$$
 by using the commutation relations :
 $[A,A^{\dagger}]=1 , [\Sigma_z,\Sigma_{\pm}]=\pm\Sigma_{\pm}, [\Sigma_+,\Sigma_-]=2\Sigma_z$
 
 $$\dot{a}=-(\gamma_c-i\Delta_{pc})a-\frac{ig_1p_+}{2}$$
 
 $$\dot{p_+}=-(\gamma_2+\frac{i\Delta_{pa}}{2})p_+-\frac{i\omega_1p_z}{2}-ig_1ap_z$$
 
 $$\dot{p_z}=-\gamma_1(p_z-p_0)+\frac{ig_1(ap^*_+-p_+a^*)}{2}+\frac{i\omega_1(p^{*}_+-p_+)}{4}$$
 
 (for example the first equation is derived from the commutation between A and H,
 <br>$[A,\Sigma_{z/+/-}]=0,[A,A^{\dagger}A]=A,
 [A,A\Sigma_-]=0,[A,\Sigma_+A^{\dagger}]=\Sigma_+$\)
 <br>
 consider the tarnsformation :
 $$\begin{pmatrix} 
\Sigma'_+  \\
\Sigma'_-  \\
\Sigma'_z  \\
\end{pmatrix}
=M_a
\begin{pmatrix} 
\Sigma_+  \\
\Sigma_-  \\
\Sigma_z  \\
\end{pmatrix}
$$
where
$$M_a=\begin{pmatrix} 
\frac{sin\alpha+1}{2} & \frac{sin\alpha-1}{2} & -cos\alpha \\
\frac{sin\alpha-1}{2} & \frac{sin\alpha+1}{2} & -cos\alpha\\
\frac{cos\alpha}{2} & \frac{cos\alpha}{2} & sin\alpha \\
\end{pmatrix}$$
and $tan\alpha=(-\Delta_{pa}/\omega_1)$.
<br>
additionaly the transformed operators satisfy the commutation relations :
 $ [\Sigma'_z,\Sigma'_{\pm}]=\pm\Sigma'_{\pm}, [\Sigma'_+,\Sigma'_-]=2\Sigma'_z$
<br>
The new equations of motion are:
 $$\dot{a}=-(\gamma_c-i\Delta_{pc})a-\frac{ig'_1p'_+}{2}$$
 
 $$\dot{p'_+}=-(\gamma'_2+\frac{i\omega_R}{2})p'_+-ig'_1ap'_z$$
 
 $$\dot{p'_z}=-\gamma'_1(p'_z-p'_0)+\frac{ig'_1(ap'^*_+-p'_+a^*)}{2}$$
 
 